In [11]:
import pandas as pd

In [12]:
RAW_DATASET_FOLDER = '../data/raw'
PROCESSED_DATASET_FOLDER = '../data/processed'
DATASET_LOCATION = f'{RAW_DATASET_FOLDER}/Language Detection.csv'
DATASET_CLEAN_LOCATION = f'{PROCESSED_DATASET_FOLDER}/Language Detection Clean.csv'
DATASET_CLEAN_UNDERSAMPLING_LOCATION = f'{PROCESSED_DATASET_FOLDER}/Language Detection Clean Undersampling.csv'
df = pd.read_csv(DATASET_LOCATION)

In [13]:
df_iteng = df[(df['Language'] == 'Italian') | (df['Language'] == 'English')].copy()

In [14]:
# Per prima cosa metto tutto in minuscolo. Poi sostituisco l'apostrofo con uno spazio (Altrimenti costrutti come l'uomo diventavano un unico token luomo)
# Successivamente elimino i numeri e poi i caratteri non alfanumerici a parte gli spazi. Elimino infine gli spazi multipli in modo da avere solo uno spazio che divide le parole
def clean_text(text):
    text = text.lower()
    text = text.replace("'", " ")
    text = ''.join(c for c in text if not c.isdigit())
    text = ''.join(c for c in text if c.isalnum() or c.isspace())
    text = ' '.join(text.split())
    return text

In [15]:
df_iteng['Text_clean'] = df_iteng['Text'].apply(clean_text)

In [16]:
# Divido in token le frasi 
df_iteng['Tokens'] = df_iteng['Text_clean'].apply(lambda x: x.split())
df_iteng['Word_count'] = df_iteng['Text_clean'].apply(lambda x: len(str(x).split()))

In [17]:
df_clean = df_iteng[["Text_clean", "Language", "Tokens"]]
df_clean = df_clean.rename(columns={"Text_clean": "Text"})

In [18]:
df_clean.to_csv(DATASET_CLEAN_LOCATION, index=False)

In [19]:
## Undersampling utilizzando i quartili

english_df = df_iteng[df_iteng['Language'] == 'English'].copy()
english_df['quartile'] = pd.qcut(english_df['Word_count'], q=4, labels=False)

undersampled_parts = []
for q in range(4):
    group = english_df[english_df['quartile'] == q]
    sampled = group.sample(frac=0.5, random_state=1999)
    undersampled_parts.append(sampled)
english_undersampled = pd.concat(undersampled_parts)
italian_df = df_iteng[df_iteng['Language'] == 'Italian']
df_clean_undersampling = pd.concat([english_undersampled, italian_df]).reset_index(drop=True)

In [20]:
df_clean_undersampling = df_clean_undersampling[["Text_clean", "Language", "Tokens"]]
df_clean_undersampling = df_clean_undersampling.rename(columns={"Text_clean": "Text"})

In [21]:
df_clean.to_csv(DATASET_CLEAN_UNDERSAMPLING_LOCATION, index=False)